In [7]:
import numpy as np
import pandas as pd
import pickle
import calendar
from datetime import datetime    

In [2]:
def days_in_next_month():
    # Get the current year and month
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month

    # Determine the next month
    next_month = current_month + 1
    next_year = current_year

    # If the month is December, the next month will be January of the next year
    if next_month > 12:
        next_month = 1
        next_year += 1

    # Get the number of days in the next month
    days_in_next_month = calendar.monthrange(next_year, next_month)[1]

    return days_in_next_month

In [9]:
STATION_ID = 611

## Import Data

In [19]:
mta_subway_df = pd.read_csv('mta-subway-transformed.csv', parse_dates=[6])
mta_subway_df = mta_subway_df.sort_values(by=["created_date"], ascending=True)
mta_subway_df.set_index('created_date', inplace=True)
mta_subway_by_station = mta_subway_df[mta_subway_df['station_complex_id'] == STATION_ID][['station_complex_id', 'number_of_riderships']]

## Upload Model

In [5]:
with open('sarimax_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)b

## Make Predictions

In [20]:
forecast_steps = days_in_next_month()
forecast_obj = loaded_model.get_forecast(steps=forecast_steps)
forecast_mean = forecast_obj.predicted_mean
conf_int = forecast_obj.conf_int(alpha=0.5)

forecasts = pd.concat([forecast_mean, conf_int], axis=1)
forecasts.columns = ['number_of_riderships', 'lower_bound', 'upper_bound']
all_forecasts = pd.concat([mta_subway_by_station, forecasts], axis=0).reset_index()

In [21]:
all_forecasts.tail()

,index,station_complex_id,number_of_riderships,lower_bound,upper_bound
665,2024-10-27,NaN,38807.218322,16901.799654,60712.636989
666,2024-10-28,NaN,73089.648210,50790.343887,95388.952533
667,2024-10-29,NaN,96593.770664,73808.144003,119379.397326
668,2024-10-30,NaN,100113.819141,76901.735685,123325.902598
669,2024-10-31,NaN,96688.783103,73067.085386,120310.480821


## Transform Dataset

In [22]:
all_forecasts.to_csv('all_forecasts.csv', index=False)